# Playbook to showcase KubeHound against our demo kind cluster

This notebook (and the ones next to it in this folder) will be in sync between the docker container and the hosts.
You can use this to experiment and save your queries in git, if needed.
This file is persistant.


## Initial Setup

Connection is being initated directly from the docker using the env vars `GRAPH_NOTEBOOK_HOST` and `GRAPH_NOTEBOOK_PORT`. To overwrite it you can use the magic `%%graph_notebook_config` [details here](https://github.com/aws/graph-notebook/tree/main/additional-databases/gremlin-server#connecting-to-a-local-gremlin-server-from-jupyter).

Now set the appearance customizations for the notebook. You can see a guide on possible options [here](https://github.com/aws/graph-notebook/blob/623d43827f798c33125219e8f45ad1b6e5b29513/src/graph_notebook/notebooks/01-Neptune-Database/02-Visualization/Grouping-and-Appearance-Customization-Gremlin.ipynb#L680)

In [ ]:
%%capture "Remove this line to see debug information"
%%graph_notebook_vis_options
{
  "edges": {
    "smooth": {
      "enabled": true,
      "type": "dynamic"
    },
    "arrows": {
      "to": {
        "enabled": true,
        "type": "arrow"
      }
    }
  }
}

### What are we looking at ?

Let's see the big pictures of our cluster.

In [ ]:
%%gremlin -d label -g class -le 50 -p inv,oute
kh.V().path().by(elementMap())

In [ ]:
%%gremlin -d label -g class -le 50 -p inv,oute
kh.V().criticalPaths().by(elementMap()).limit(2000)

Well, this is bad. This cluster is definitely not secured. We need to focus on what make actually sense from a real attacker perspective. 

### Let's look at the containers then

In [ ]:
%%gremlin -d label -g class -le 50 -p inv,oute
kh.containers().criticalPaths().by(elementMap())

Nop, still too much result, let's try to narrow it down. What is more important than the containers ? Endpoint, because they are the component that actually exposed service.
Supply chain attack are great but they are sophisticated. 

### Let's look at the basic, the endpoints

In [ ]:
%%gremlin -d name -g class -le 50 -p inv,oute
kh.endpoints().criticalPaths().by(elementMap())

OK this is making more sense, let's get the context of the associated endpoint that leads to a critical attack path.

### Let's get the vulnerability context

In [ ]:
%%gremlin -d name -g class -le 50 -p inv,oute
kh.endpoints().criticalPaths().limit(local,1)
.dedup().valueMap("serviceEndpoint","port")
.group().by("serviceEndpoint").by("port")

OK so all are not equal, and some are not in our interest. Let's remove the `kube-dns` service.

### Let's look at the specific services that interest us

In [ ]:
%%gremlin -d label -g class -le 50 -p inv,oute
kh.endpoints().not(has("serviceEndpoint","kube-dns")).criticalPaths().by(elementMap())

Ok, so once we land on the node, it is game over. This will require too much in-depth security. So let's focus on the first initial stage first.

### Initial exposure identified

In [ ]:
%%gremlin -d label -g class -le 50 -p inv,oute
kh.endpoints().not(has("serviceEndpoint","kube-dns"))
	.repeat(
		outE().inV().simplePath()
	)
	.until(
		hasLabel("Node")
		.or()
		.loops().is(5)
	)
	.hasLabel("Node")
	.path()
	.by(elementMap())
	.limit(100)	// Limit the number of results for large clusters

Here we have, the "true" findings that need our attention.